In [1]:
import threading
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import time
import re
import serial
import sys
import glob
import serial
import numpy as np

In [2]:
#https://stackoverflow.com/questions/12090503/listing-available-com-ports-with-python
def get_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [3]:
def list_ports():
    print('Available serial ports: ')
    ports=get_ports()
    for port in ports:
        print(ports.index(port), end=':')
        print(port)

In [4]:
def try_connect(portindex):
    myport = None
    try:
        myport = serial.Serial(
            port=get_ports()[portindex],
            baudrate=250000,
            timeout=1.0,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS
        )
        print('Sucessfully connected to port '+myport.port)
        return myport
    except (OSError, serial.SerialException):
        print('ERROR: ', end='')
        if myport.is_open:
            print('Port already open')
        elif myport.port not in portlist:
            print('Port '+myport.port+' not found')
        else:
            print('Could not connect to port')
        pass

In [13]:
def isCoordinateList(printeroutput):
    if(len(re.findall(xyzregexpattern, printeroutput))==3):
        return True
    return False
xyzregexpattern = "(?:[^E][:])(\d{1,3}[.]\d{4})"
def parse_coordinates(printeroutput):
    return list(map(float, re.findall(xyzregexpattern,printeroutput)))

In [31]:
def handle_input(sender):
    sender.disabled=True
    userinput=sender.value
    if userinput in macros:
        with terminalout:
            print("MACRO: ",userinput)
        macros[userinput]()

    else:
        send_gcode(userinput)
    wait_for_ok()
    #myport.reset_input_buffer()
    sender.disabled=False
    sender.value=''

def read_serial():
    return list(map(lambda line: line.decode().rstrip(), myport.readlines()))

def contains_ok(lines):
    if 'ok' in lines:
        return True
    return False
        

def send_gcode(gcode, wait=True):
    raw = gcode.upper()
    with terminalout:
        print('SEND: ' + raw); #remove trailing whitespace and print
    sInput = (raw + lineTerminator) #append terminator string
    bInput = sInput.encode() #convert string to bytes
    myport.write(bInput) #send the command


def wait_for_ok(timeout=10000):
    waitend = current_milli_time()+timeout
    while(current_milli_time()<waitend):
        lines = read_serial()
        for line in lines:
            with terminalout:
                print('RECV: '+ line)
        if(not contains_ok(lines)):
            time.sleep(0.01)
        else:
            break
    if(current_milli_time()>=waitend):
        with terminalout:
            print("OK not received, printer may still be busy")

def current_milli_time():
    return round(time.time() * 1000)

In [28]:
## String constants ##
commandPrompt='Command: '
lineTerminator='\r\n'

## MACRO DEFINITIONS ##
def show_help():
    """Shows this help"""
    with terminalout:
        print('Available macros:')
        for macro in macros.items():
            print("'"+macro[0]+"'", end=': ')
            print(macro[1].__doc__)
def probe_point(self):
    """Run probing routine"""
    print('probe point macro')
def home():
    """Auto Home XYZ"""
    send_gcode('G28', False)
    return
def deploy():
    """Deploy the probe"""
    send_gcode('M401')
    return
def stow():
    """Stow the probe"""
    send_gcode('M402')
    return
def fan_on():
    """Set case fan speed to MAX"""
    send_gcode('M106 P0 S255')
    return
def fan_off():
    """Disable all fans"""
    send_gcode('M107')
    return
def get_pos():
    """Report the current tool position"""
    send_gcode('M114')
    return

## MACRO DICTIONARY
macros = {
    'help':show_help,
    'probe point':probe_point,
    'home':home,
    'deploy':deploy,
    'stow':stow,
    'fanon':fan_on,
    'fanoff':fan_off,
    'getpos':get_pos
}


In [29]:
terminalout = widgets.Output(
    layout={
            'width':'auto%',
            'height':'500px',
            'border': '1px solid white',
            'overflow':'scroll',
           }
)
userin = widgets.Text(
    layout={
            'width':'auto%',
            'border': '1px solid white'
           },
    value='',
    placeholder='Type help for available macros',
    description=commandPrompt,
    disabled=False
)
userin.on_submit(handle_input)
box_layout = {'display':'flex',
                    'flex_flow':'column',
                    'align_items':'stretch',
                    'border':'solid',
                    'width':'auto%'}
termbox = VBox([terminalout, userin])

In [9]:
list_ports()

Available serial ports: 
0:COM1
1:COM5
2:COM12
3:COM14


In [10]:
myport=try_connect(1)

Sucessfully connected to port COM5


In [32]:
myport.reset_input_buffer()
terminalout.clear_output()
display(termbox)
print('Enter your commands in the box above. Press <ENTER> to send.')
print('Type "help" to see available shorcuts')
print('\nNOTE:\tAfter a command is sent, the input field will be disabled\n\tuntil an OK is received or the timeout period has elapsed')

Enter your commands in the box above. Press <ENTER> to send.
Type "help" to see available shorcuts

NOTE:	After a command is sent, the input field will be disabled
	until an OK is received or the timeout period has elapsed
